In [ ]:
import pandas as pd
BASE_URL = "https://voyager.online/api/staking/validators"
page_size = 50

# Iterate over pages 1 to 3 (adjust the range as needed)
for p in range(1, 4):
    url = f"{BASE_URL}?search=&p={p}&ps={page_size}"
    data = requests.get(url).json()
    print(f"Page {p}:")
    print(data)


In [4]:
import requests
import psycopg2

conn = psycopg2.connect(
    dbname="mydatabase",
    user="user",
    password="password",
    host="localhost",
    port="5432"  # typically 5432
)
cur = conn.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS validators (
    address TEXT PRIMARY KEY,
    tokenAddress TEXT,
    isVerified BOOLEAN,
    totalStake NUMERIC,             -- changed from TEXT to NUMERIC
    totalStakePercentage NUMERIC,   -- changed from INTEGER to NUMERIC if decimals are needed
    ownStake NUMERIC,               -- changed from TEXT to NUMERIC
    delegatedStake NUMERIC,         -- changed from TEXT to NUMERIC
    totalDelegators INTEGER,
    revenueShare NUMERIC,           -- changed from INTEGER to NUMERIC if needed
    apr NUMERIC,                    -- changed from INTEGER to NUMERIC if decimals are needed
    rank TEXT,
    name TEXT,
    imgSrc TEXT,
    startTime BIGINT,
    poolAddress TEXT,
    stakerAddress TEXT
);
"""
cur.execute(create_table_query)
conn.commit()

BASE_URL = "https://voyager.online/api/staking/validators"
page_size = 100

for p in range(1, 4):
    url = f"{BASE_URL}?search=&p={p}&ps={page_size}"
    response = requests.get(url)
    data = response.json()
    items = data.get("items", [])
    
    for item in items:
        query = """
        INSERT INTO validators (
            address, tokenAddress, isVerified, totalStake, totalStakePercentage,
            ownStake, delegatedStake, totalDelegators, revenueShare, apr, rank,
            name, imgSrc, startTime, poolAddress, stakerAddress
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (address) DO NOTHING;
        """
        values = (
            item.get("address"),
            item.get("tokenAddress"),
            item.get("isVerified"),
            item.get("totalStake"),
            item.get("totalStakePercentage"),
            item.get("ownStake"),
            item.get("delegatedStake"),
            item.get("totalDelegators"),
            item.get("revenueShare"),
            item.get("apr"),
            item.get("rank"),
            item.get("name"),
            item.get("imgSrc"),
            item.get("startTime"),
            item.get("poolAddress"),
            item.get("stakerAddress")
        )
        cur.execute(query, values)
    
    conn.commit()
    print(f"Inserted data from page {p}")

cur.close()
conn.close()

Inserted data from page 1
Inserted data from page 2
Inserted data from page 3


In [3]:
import requests
import psycopg2


conn = psycopg2.connect(
    dbname="mydatabase",
    user="user",
    password="password",
    host="localhost",
    port="5432" 
)
cur = conn.cursor()

query_get_avg_delegated_stake_top_10 = """SELECT AVG(delegatedStake) 
FROM (
    SELECT delegatedStake 
    FROM validators 
    ORDER BY delegatedStake DESC
    limit 10
) subquery;"""


cur.execute(query_get_avg_delegated_stake_top_10)
result = cur.fetchall()
print(result[0][0] / 10**18)

cur.close()
conn.close()


15209196.437081446278959945


In [12]:
import requests

selector = "0x1e888a1026b19c8c0b57c72d63ed1737106aa10034105b980ba117bd0c29fe1"

RPC_URL = "https://free-rpc.nethermind.io/mainnet-juno/v0_7"

contract_address = "0x04718f5a0fc34cc1af16a1cdee98ffb20c31f5cd61d6ab07201858f4287c938d"
owner = "0x030016e7beABe590593a1958e31D2aFa2D06754Ea8468e3e720c781c3cA690d6"
spender = "0x071c4334c5b387cbc84a3a62b16c6eb0481bf596054d18fd983e5f3fc27e489b"

payload = {
    "jsonrpc": "2.0",
    "method": "starknet_call",
    "params": [
        {
            "contract_address": contract_address,
            "entry_point_selector": selector,
            "calldata": [owner, spender]
        },
        "latest"
    ],
    "id": 1
}

r = requests.post(RPC_URL, json=payload)
result = r.json()
print(result)

if "result" in result:
    allowance_hex = result["result"][0]
    print("Allowance (hex):", allowance_hex)
    print("Allowance (dec):", int(allowance_hex, 16))
else:
    print("Error:", result.get("error"))


{'jsonrpc': '2.0', 'result': ['0xde0b6b3a7640000', '0x0'], 'id': 1}
Allowance (hex): 0xde0b6b3a7640000
Allowance (dec): 1000000000000000000


In [1]:
import requests
import pandas as pd


BASE_URL = "https://voyager.online/api/staking/delegators?validator=0x036963c7b56f08105ffdd7f12560924bdc0cb29ce210417ecbc8bf3c7e4b9090"
page_size = 100
all_items = []

for p in range(1, 55):
    # Append pagination parameters to the URL
    url = f"{BASE_URL}&search=&p={p}&ps={page_size}"
    print("Requesting URL:", url)
    
    response = requests.get(url)
    

    try:
        data = response.json()
        items = data.get("items", [])
        print(f"Page {p}: Retrieved {len(items)} items")
        all_items.extend(items)
    except Exception as e:
        print(f"Error parsing JSON response on page {p}:", e)

# Convert the aggregated items into a DataFrame
df = pd.DataFrame(all_items)

# Display some information about the DataFrame
print("DataFrame shape:", df.shape)
print(df.head())


Requesting URL: https://voyager.online/api/staking/delegators?validator=0x036963c7b56f08105ffdd7f12560924bdc0cb29ce210417ecbc8bf3c7e4b9090&search=&p=1&ps=100
Page 1: Retrieved 100 items
Requesting URL: https://voyager.online/api/staking/delegators?validator=0x036963c7b56f08105ffdd7f12560924bdc0cb29ce210417ecbc8bf3c7e4b9090&search=&p=2&ps=100
Page 2: Retrieved 100 items
Requesting URL: https://voyager.online/api/staking/delegators?validator=0x036963c7b56f08105ffdd7f12560924bdc0cb29ce210417ecbc8bf3c7e4b9090&search=&p=3&ps=100
Page 3: Retrieved 100 items
Requesting URL: https://voyager.online/api/staking/delegators?validator=0x036963c7b56f08105ffdd7f12560924bdc0cb29ce210417ecbc8bf3c7e4b9090&search=&p=4&ps=100
Page 4: Retrieved 100 items
Requesting URL: https://voyager.online/api/staking/delegators?validator=0x036963c7b56f08105ffdd7f12560924bdc0cb29ce210417ecbc8bf3c7e4b9090&search=&p=5&ps=100
Page 5: Retrieved 100 items
Requesting URL: https://voyager.online/api/staking/delegators?validato

In [ ]:
import requests
import psycopg2

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname="mydatabase",
    user="user",
    password="password",
    host="localhost",
    port="5432"  # typically 5432
)
cur = conn.cursor()

# Create the delegators table if it doesn't exist
create_table_query = """
CREATE TABLE IF NOT EXISTS delegators (
    address TEXT,
    delegatedStake NUMERIC,
    share NUMERIC,
    startTime BIGINT,
    unpoolTime BIGINT,
    withdrawalTime BIGINT,
    delegatedTo TEXT,
    PRIMARY KEY (address, delegatedTo)
);
"""
cur.execute(create_table_query)
conn.commit()
print("Created (or verified) the delegators table.")

# Get all validator addresses from the validators table
cur.execute("SELECT address FROM validators;")
validator_rows = cur.fetchall()
validator_addresses = [row[0] for row in validator_rows]
print("Retrieved validator addresses:", validator_addresses)

# Base URL for the delegators endpoint
BASE_URL = "https://voyager.online/api/staking/delegators"
page_size = 100

# Iterate over every validator address and process pagination
for validator in validator_addresses:
    print(f"\nProcessing validator: {validator}")
    
    # Iterate through pages until no items are returned
    for p in range(1, 55):
        url = f"{BASE_URL}?validator={validator}&search=&p={p}&ps={page_size}"
        print("Requesting URL:", url)
        response = requests.get(url)
        
        try:
            data = response.json()
            items = data.get("items", [])
            print(f"Page {p}: Retrieved {len(items)} items for validator {validator}")
            
            # Break out of pagination loop if no more items
            if not items:
                break
            
            for item in items:
                query = """
                INSERT INTO delegators (
                    address, delegatedStake, share, startTime, unpoolTime, withdrawalTime, delegatedTo
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                ON CONFLICT (address, delegatedTo) DO NOTHING;
                """
                values = (
                    item.get("address"),
                    item.get("delegatedStake"),
                    item.get("share"),
                    item.get("startTime"),
                    item.get("unpoolTime"),
                    item.get("withdrawalTime"),
                    validator  
                )
                cur.execute(query, values)
            conn.commit()
        except Exception as e:
            print(f"Error processing page {p} for validator {validator}:", e)

cur.close()
conn.close()
print("ETL process completed.")


Created (or verified) the delegators table.
Retrieved validator addresses: ['0x00d3b910d8c528bf0216866053c3821ac6c97983dc096bff642e9a3549210ee7', '0x072543946080646d1aac08bb4ba6f6531b2b29ce41ebfe72b8a6506500d5220e', '0x036963c7b56f08105ffdd7f12560924bdc0cb29ce210417ecbc8bf3c7e4b9090', '0x04b00f97e2d2168b91fe64ceeace4a41fc274a85bbdd0adc402c3d0cf9f91bbb', '0x033d152598873a307b9abdd7bd5d479cadf1814db4bf87e0f0d61a8f84d0bf63', '0x051ba9be967d17aaafac92f9bc7ca4b035dfd3c4a97b32be1773f63e27b0526a', '0x064204710398bbd2a3af22b9335f4d75234f16e5badb5210b686232f2333d9f1', '0x015cc95c70120ec985bdd6115232dd5f875bd58820aa4c62b6565d0fe33c3ac4', '0x070ff78577847c55027d6defb5f7613714e052e47235fe8e7bac7c0915a8e08f', '0x043a242b45c03683e2acc96bb7d207c0cfec6d554b224c76c3b690960b485adb', '0x0226bdb0501a92f5980e865124d99bf2bba33de9556c2961cc6818ed13c0dff9', '0x077d4b4e7ae321aabd0a5a7322108635fcbd0cd746f9ae217b8ea00363494b65', '0x071768d9d5468cac52c7f4b0f1265fecd7cb1a9733dfa453a27aaf900a6946a8', '0x01aca15766c